In [1]:
import tflscripts
from tflscripts import take_percentage_of_data, read_dataset, filter_by_features, X_sort, filter_by_activities_transfer, classify, easy_domain_adaptation_update_dataframes, take_multiple_percentages_of_data, concat_and_reindex, filter_by_activities
from sklearn.metrics import accuracy_score
import numpy as np

ImportError: No module named tflscripts

In [73]:
source_dataset = 'synergy-final-iter1'
source_device = 'xdk_1'
target_dataset = 'robotics-final'
target_device = 'xdk_2'

use_features = 'accel_'
force_columns = None
use_columns = None
use_activities = [0,1,2,3,4,5,7]
use_target_training_activities = [0,1,2]
activities_to_test = [0,1,2]
with_feature_selection = False
scale_domains_independently = False
use_easy_domain_adaptation = True
clf_name = 'LinearSVC'

training_source_data_ratio = 0.6
training_target_data_ratio = 0.6
testing_target_data_ratio = 0.4

tflscripts.set_dataset_folder('/home/giotto/transfer-learning-playground/datasets/')

In [74]:
# read datasets
df_source, df_source_labels = read_dataset(
        source_dataset,
        source_device,
        with_feature_selection=with_feature_selection)
df_target, df_target_labels = read_dataset(
        target_dataset,
        target_device,
        with_feature_selection=with_feature_selection)

# filter features
if use_features is not None:
    df_source, df_target = filter_by_features(df_source=df_source,
                                              df_target=df_target,
                                              use_features=use_features)
    if df_source is None:
        print('No features found')

if force_columns is not None:
    use_columns = force_columns
    add_empty_columns_if_missing(df_source, force_columns)
    add_empty_columns_if_missing(df_target, force_columns)

# filter specific columns
if use_columns is not None:
    try:
        df_source = df_source[use_columns]
        df_target = df_target[use_columns]
    except KeyError as ex:
        print('No such columns found')

# filter activities
if use_activities is not None:
    df_source, df_source_labels, df_target, df_target_labels = \
        filter_by_activities_transfer(
            df_source, df_source_labels, df_target,
            df_target_labels, use_activities)
    if df_source is None:
        print('No activities found')

# do easy domain adaptation
if use_easy_domain_adaptation:
    df_source, df_target = easy_domain_adaptation_update_dataframes(
            df_source, df_target)

# filter samples
df_source, df_source_labels = take_percentage_of_data(
        df_source,
        df_source_labels,
        training_source_data_ratio)

# split the target data to testing and training if necessary
if training_target_data_ratio > 0.0:
    dfs_target = take_multiple_percentages_of_data(
            df_target,
            df_target_labels,
            [training_target_data_ratio, testing_target_data_ratio])

    df_target_train, df_target_train_labels = dfs_target[0]
    df_target, df_target_labels = dfs_target[1]
    
    # filter target training activities
    if use_target_training_activities is not None:
        df_target_train, df_target_train_labels = \
            filter_by_activities(
                df_target_train, df_target_train_labels, use_target_training_activities)

    df_source, df_source_labels = concat_and_reindex(
            [df_source, df_target_train],
            [df_source_labels, df_target_train_labels])
else:
    df_target, df_target_labels = take_percentage_of_data(
            df_target,
            df_target_labels,
            testing_target_data_ratio)

if activities_to_test is not None:
    df_target, df_target_labels = \
            filter_by_activities(df_target, df_target_labels, activities_to_test)

# sort feature columns
X_source = X_sort(df_source)
X_target = X_sort(df_target)

# scale domains
if scale_domains_independently:
    df_source = StandardScaler().fit_transform(df_source)
    df_target = StandardScaler().fit_transform(df_target)

y_source = df_source_labels['label']
y_target = df_target_labels['label']

accuracies = []

for i in range(20):
    y_target_pred = classify(X_source, y_source, X_target, clf_name,
                             scale=not scale_domains_independently)

    accuracies.append(accuracy_score(y_target, y_target_pred))

np.mean(accuracies)

0.36263736263736257